In [1]:
import marqo
import sys
import pathlib
import pickle

In [2]:
import numpy as np

In [3]:
sys.path.insert(0, "src")
import data_functions
import embedding_functions
import cluster_functions
import graph_functions
import util_functions
from MarqoExternalUtils import MarqoEmbeddingStore, MarqoDocumentStore, MarqoDocument

C:\Users\fra3066mat\PycharmProjects\concept-graphs\src\data_functions.py:14: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


base_name_test = "grascco_lokal_test"
path_name_test = lambda x: f"./tmp/{base_name_test}/{base_name_test}_{x}.pickle"---

In [4]:
base_name = "grascco_lokal"
path_name = lambda x: f"./tmp/{base_name}/{base_name}_{x}.pickle"

In [5]:
base_name_test = "grascco_lokal_test"
path_name_test = lambda x: f"./tmp/{base_name_test}/{base_name_test}_{x}.pickle"

In [6]:
grascco_data_test = data_functions.DataProcessingFactory.load(
    file_path=path_name_test("data"),
)

In [7]:
grascco_embedding_test = embedding_functions.SentenceEmbeddingsFactory.create(
    data_obj=grascco_data_test,
    cache_path=pathlib.Path(path_name_test("embedding")).parent,
    cache_name="_".join(pathlib.Path(path_name_test("embedding")).stem.split("_")[:-1]),
    model_name="Sahajtomar/German-semantic",
    storage_method=("vector_store", {},),
    vectorstore_normalizeEmbeddings=False,
    vectorstore_annParameters={
        "spaceType": "dotproduct", #prenormalized-angular(default)
        "parameters": {
            "efConstruction": 1024,
            "m": 16
        }
    }
)

INFO:root:Creating Sentence Embedding with 'None'
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: Sahajtomar/German-semantic
Batches: 100%|██████████| 28/28 [00:11<00:00,  2.44it/s]
2025-05-22 19:08:51,765 logger:'marqo' WARNING Your Marqo Python client requires a minimum Marqo version of 2.18.0 to function properly, but your Marqo version is 2.16.1. Please upgrade your Marqo instance to avoid potential errors. If you have already changed your Marqo instance but still get this warning, please restart your Python interpreter.
2025-05-22 19:08:57,535 logger:'marqo' INFO     add_documents batch 0: took 5.541s for Marqo to process & index 128 docs. Roundtrip time: 5.723s.
INFO:marqo:    add_documents batch 0: took 5.541s for Marqo to process & index 128 docs. Roundtrip time: 5.723s.
2025-05-22 19:08:58,415 logger:'marqo' INFO     add_documents batch 1: took 0.694s for Marqo to pr

Saved under: C:\Users\fra3066mat\PycharmProjects\concept-graphs\tmp\grascco_lokal_test\grascco_lokal_test.pickle


In [10]:
grascco_embedding = embedding_functions.SentenceEmbeddingsFactory.load(
    data_obj_path=path_name("data"),
    embeddings_path=path_name("embedding")
)

In [ ]:
grascco_embedding_from_vec = embedding_functions.SentenceEmbeddingsFactory.load(
    data_obj_path=path_name("data"),
    embeddings_path=path_name("vec"),
    storage_method=("vector_store", {},),
)

In [ ]:
grascco_clustering = cluster_functions.PhraseClusterFactory.load(
    data_obj_path=path_name("clustering"),
)

In [8]:
grascco_graph_test = pickle.load(
    open(path_name_test("graph"), 'rb')
)

In [ ]:
grascco_data = data_functions.DataProcessingFactory.load(
    file_path=path_name("data"),
)

---

In [11]:
embedded_phrases = [
    (chunk["text"], grascco_embedding.sentence_embeddings[idx]) for idx, chunk in enumerate(grascco_embedding.data_processing_obj.data_chunk_sets)
    if 8 in [d["id"] for d in chunk["doc"]]
]

In [12]:
phrases, embeddings = zip(*embedded_phrases)
test_doc = MarqoDocument(
    phrases=phrases,
    embeddings=embeddings,
)

---

In [13]:
mqs = MarqoEmbeddingStore("http://localhost:8882", "grascco_lokal_test")
mqds = MarqoDocumentStore(embedding_store=mqs)

In [14]:
mqs.best_hits_for_field("876")

[]

In [19]:
mqds.add_document(test_doc)

2025-05-22 19:10:37,665 logger:'marqo' INFO     add_documents batch 0: took 0.568s for Marqo to process & index 116 docs. Roundtrip time: 0.735s.
INFO:marqo:    add_documents batch 0: took 0.568s for Marqo to process & index 116 docs. Roundtrip time: 0.735s.
INFO:root:For id '877' the same embedding is already in the index with id '0'.
INFO:root:For id '878' the same embedding is already in the index with id '1'.
INFO:root:For id '879' the same embedding is already in the index with id '49'.
INFO:root:For id '880' the same embedding is already in the index with id '54'.
INFO:root:For id '881' the same embedding is already in the index with id '65'.
INFO:root:For id '882' the same embedding is already in the index with id '152'.
INFO:root:For id '883' the same embedding is already in the index with id '190'.
INFO:root:For id '884' the same embedding is already in the index with id '210'.
INFO:root:For id '885' the same embedding is already in the index with id '211'.
INFO:root:For id '8

MarqoWebError: MarqoWebError: MarqoWebError Error message: {'message': '[{"loc": ["__root__"], "msg": "Custom vector field \'vector\' must be a list, but given vector is of type <class \'NoneType\'>", "type": "value_error"}]', 'code': 'invalid_argument', 'type': 'invalid_request', 'link': ''}
status_code: 400, type: invalid_request, code: invalid_argument, link: 

In [ ]:
mqs.marqo_index.recommend(["10"], exclude_input_documents=False)

In [18]:
mqs.marqo_index.get_document("10")

{'graph_cluster': ['2'], '_id': '10', 'phrase': 'handamputation'}

In [ ]:
mqs.store_size

In [ ]:
mqs.marqo_index.get_document("87")

In [17]:
for i, g in enumerate(grascco_graph_test):
    doc_list = []
    for n, d in g.nodes(True):
        doc_list.append(
            {
                "_id": str(n),
                "graph_cluster": [str(i)]
            }
        )
    _res = mqs.marqo_index.update_documents(
        documents=doc_list,
        # client_batch_size=128,
        # use_existing_tensors=True
    )

In [ ]:
doc_list

---

In [ ]:
grascco_embedding.data_processing_obj.data_chunk_sets[1]

In [ ]:
vector_name = "phrase_vector"
def create_index_document(sentence_embedding_obj: 'embedding_functions.SentenceEmbeddingsFactory.SentenceEmbeddings'):
    for i, embedding in enumerate(sentence_embedding_obj.sentence_embeddings):
        _chunk_obj = sentence_embedding_obj.data_processing_obj.data_chunk_sets[i]
        yield {
            "_id": str(i),
            "documents": [str(d['id']) for d in _chunk_obj['doc']],
            #"phrase": _chunk_obj['text'],
            vector_name: {
                "content": _chunk_obj['text'],
                "vector": [float(f) for f in embedding]
            }
        }

In [ ]:
mq = marqo.Client(url="http://localhost:8882")

In [ ]:
index_name = "grascco_index"
settings = {
    "treatUrlsAndPointersAsImages": False,
    "model": "no_model",
    "modelProperties": {
        "dimensions": 1024,
        "type": "no_model",
    },
    "annParameters": {
        "spaceType": "dotproduct", #prenormalized-angular(default)
        "parameters": {
            "efConstruction": 1024,
            "m": 16
        }
    }
}
try:
    mq.create_index(
        index_name=index_name,
        settings_dict=settings,
    )
except:
    mq.delete_index(index_name)
    mq.create_index(
        index_name=index_name,
        settings_dict=settings,
    )

In [ ]:
mq.index(index_name).add_documents(
    list(create_index_document(grascco_embedding)),
    client_batch_size=128,
    tensor_fields=[vector_name],
    mappings={
        vector_name: {
            "type": "custom_vector"
        }
    },
)

In [ ]:
mq.index("grascco_lokal_test").get_document("877", expose_facets=True)

In [ ]:
mqs.store_embedding(("handamputation", mq.index("grascco_lokal_test").get_document("10", expose_facets=True)["_tensor_facets"][0]["_embedding"]))

In [ ]:
mqs.best_hits_for_field("10")

In [ ]:
mqs.marqo_index.delete_documents(["877"])

In [ ]:
mq.index("grascco_lokal_test").get_documents(["10", "11"])

In [ ]:
mq.index("grascco_lokal_test").recommend(
    documents=['877'],
    tensor_fields=["phrase_vector"],
    limit=2,
    exclude_input_documents=False
).get("hits")

In [ ]:
ex_doc = mq.index("grascco_lokal_test").get_document("10", expose_facets=True)
ex_doc

In [ ]:
mq.index("grascco_lokal_test").search(
    q={
        "customVector" : {
            "vector": ex_doc["_tensor_facets"][0]["_embedding"],
            "content": ex_doc["phrase"],
        }
    },
    search_method="HYBRID"
)

In [ ]:
_ids = [str(i) for i in range(6)]
np.asarray([
    _res["_tensor_facets"][0]["_embedding"]
    for _res in
    mq.index(index_name).get_documents(_ids, expose_facets=True)["results"]
]).tolist()

In [ ]:
mq.index(index_name).get_stats()["numberOfDocuments"]

In [ ]:
mq.index(index_name).get_document("5180")

In [ ]:
marqo_store = MarqoEmbeddingExternal(
    client_url="http://localhost:8882",
    index_name="grascco_lokal_test"
)

In [ ]:
mq.index("grascco_lokal_test").search(
    q='psychose',
    search_method="LEXICAL",
)